### Save activations from all classes

In [ ]:
from tools import utils, config, trainer, parts
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import functional as F
from tools import utils

from data.cifar10 import get_test_loader

plt.style.use('fast')
PLOT_DIR = 'plots'

from robustbench.utils import load_model
from robustbench.data import load_cifar10

model_name = "Pang2022Robustness_WRN28_10"

test_loader = get_test_loader(batch_size=16)
test_dataset = test_loader.dataset
model = load_model(model_name=model_name, # Standard, Hendrycks2019Using, Pang2022Robustness_WRN28_10, Gowal2021Improving_28_10_ddpm_100m
                        dataset='cifar10',
                        threat_model='Linf').cuda()


part_manager = parts.PartManager(model)
part_manager.enable_all()

model.eval();

classes = list(range(10))
n_classes = len(classes)

### Get samples from each class
- Pick example indices for which the model gave a correct/wrong prediction for adversarial example

In [ ]:
def select_examples():
    class_size = 100
    classes_to_accumulate = [c for c in classes]
    examples = {i: [] for i in classes}
    predictions = {c: [] for c in classes}

    i = 0
    while i < len(test_dataset):
        x, y = test_dataset[i]
        
        # Skip if we don't need anymore examples in this class
        if not y in classes_to_accumulate:
            i+=1
            continue
        
        # Initial prediction must be correct
        y_pred = model(x.unsqueeze(0).cuda())
        y_pred = np.argmax(y_pred.cpu().detach().numpy())
        
        # Use examples where prediction is in specisified classes
        examples[y].append(i)
        predictions[y].append(y_pred)

        # Check if we need more examples for this class
        if len(examples[y]) == class_size:
            classes_to_accumulate.remove(y)
            if len(classes_to_accumulate) == 0:
                break
        i+=1
    return examples

examples = select_examples()

In [ ]:
print("Number of examples per class:")
[f'{k}:{len(v)}' for k, v in examples.items()]

In [ ]:
"""# Save example idx
import pickle

with open("experiments/exp16_cifar10_resnet18/example_idx_allclasses.pickle", "wb") as f:
    pickle.dump(examples, f, protocol=pickle.HIGHEST_PROTOCOL)"""

In [ ]:
# Load example idx
import pickle

with open("experiments/exp16_cifar10_resnet18/example_idx_allclasses.pickle", "rb") as f:
    examples = pickle.load(f)

### Get Activations for Adversarial and Regular Examples

In [ ]:
def get_activations(examples, part_i):
    part_output_list_adv = [[] for _ in range(n_classes)]
    part_output_list = [[] for _ in range(n_classes)]

    for class_i, class_examples in examples.items():
        class_order = classes.index(class_i)
        for example_i in class_examples:
            x, y = test_dataset[example_i]
            x = x.unsqueeze(0).cuda()
            _ = model(x)
            
            part_output = getattr(part_manager.parts[part_i].get_loss_end_layer(), trainer.SAVED_OUTPUT_NAME)
            part_output = torch.squeeze(part_output).cpu().detach().numpy()
            part_output_list[class_order].append(part_output)
        
    activations = np.array(part_output_list)
    #activations_adv = np.array(part_output_list_adv)
    return activations#, activations_adv

In [ ]:
from tqdm import trange
import os

if not os.path.exists(f"experiments/exp16_cifar10_{model_name}"):
    os.mkdir(f"experiments/exp16_cifar10_{model_name}")


for part_i in [0, 5, 10, 15, 20, 25]:

    # Get activations for correctly predicted examples
    activations = get_activations(examples, part_i)

    # i_class, i_example, i_kernel, h, w
    print(f"act shape: {activations.shape}")

    with open(f"experiments/exp16_cifar10_{model_name}/activations_allclasses_part{part_i}.npy", "wb") as f:
        np.save(f, activations)